In [8]:
if not('init' in globals()):
    import os
    os.chdir('../..')

    print("Current working directory: {0}".format(os.getcwd()))

    import torch 
    import torch.nn as nn

    from tests.utils import *

    import model.convolution
    import model.quantizer

    init = 0


In [9]:
dev = "cpu"
a = torch.randn(10,5,30,30).to(dev)
a = a.clamp(-0.5,0.5)

In [10]:
a2 = a.detach().clone()
ao = a.detach().clone()
exp = -8*torch.ones((1,5,1,1)).to(dev)
a2 = (a2/(2**(exp))).type(torch.int64)
# print(a2)


fun = lambda a:torch.ones_like(a)
fun = None

In [11]:
Conv = nn.Conv2d(5,5,3,1,padding=1,bias=True).to(dev)
Convq = model.convolution.Conv2d(5,5,3,1,padding=1,bias=True,weight_quant_bits=8,weight_quant_channel_wise=True,out_quant_bits=8,out_quant_channel_wise=False).to(dev)

outquant = model.quantizer.LinQuantExpScale(8,(-1,),1,"floor").to(dev)
inquant = model.quantizer.LinQuantExpScale(8,(-1,),1,"floor").to(dev)

Convq.weight.data = Conv.weight.detach().clone() 
Convq.bias.data = Conv.bias.detach().clone() 

In [12]:
a = ao.clone()
Convq.train()
outquant.train()
inquant.train()


a = inquant(ao.clone())
out2 = Convq((a,inquant.delta_out.log2()),fun)

pr("quant bias", Conv.bias, Convq.debug_bias)
pr("quant bias check", Conv.bias, Convq.bias)


pr("quant weights", Conv.weight, Convq.debug_weight)
pr("quant weights check", Conv.weight, Convq.weight)

quant bias diff: 0.126420795917511, mean: 0.05607088655233383
quant bias check diff: 0.0, mean: 0.0
quant weights diff: 0.0019012329867109656, mean: 0.0006686205160804093
quant weights check diff: 0.0, mean: 0.0


In [19]:
a = ao.clone()
Convq.train()
outquant.train()
inquant.train()


a = inquant(ao.clone())
# ao = a.clone()
out1 = Conv(a)
out1 = outquant(out1)
out2 = Convq((a,inquant.delta_out.log2()),fun)
pr("a",a,inquant(ao.clone()))
pr("out", out1,out2[0])
pr("weights", Convq.weight,Conv.weight )
pr("quant weights", Conv.weight, Convq.debug_weight)
pr("quant bias", Conv.bias, Convq.debug_bias)

Conv.weight.data = Convq.debug_weight.clone()
Conv.bias.data = Convq.debug_bias.clone()

print("outquant.delta_out",outquant.delta_out.view(-1))
print("out quant",out2[1].exp2().view(-1))
pr("out quant delta",outquant.delta_out.view(-1),out2[1].exp2().view(-1))


Convq.eval()
outquant.eval()
inquant.eval()
b = inquant(ao.clone())
# b = a
out3 = Convq(((b).type(torch.int32),inquant.delta_out.log2()),fun)
out3v = (out3[0].type(torch.float))*(out3[1].exp2())
# print(out3[0])
pr("out", out1,out3v)
pr("out bits", (out1/(2**out3[1])),out3[0])
pr("eval cmp", (out2[0]/(2**out3[1])),out3[0])
Convq.train();
print(Convq.out_quant.delta_out.view(-1))
print(Convq.debug_fact.view(-1))
print(Convq.debug_fact.log2().view(-1))
print(Convq.n.view(-1))
print(Convq.debug_n.view(-1))
# print(Convq.t.view(-1))
# print(Convq.bias.view(-1))


a diff: 0.0, mean: 0.0
out diff: 0.0, mean: 0.0
weights diff: 0.0018936034757643938, mean: 0.0006475171539932489
quant weights diff: 0.0, mean: 0.0
quant bias diff: 0.0, mean: 0.0
outquant.delta_out tensor([0.0157])
out quant tensor([0.0157])
out quant delta diff: 0.0, mean: 0.0
out diff: 0.0156862773001194, mean: 2.78867082670331e-06
out bits diff: 1.0, mean: 0.00017777777975425124
eval cmp diff: 1.0, mean: 0.00017777777975425124
tensor([0.0157])
tensor([0.5842, 0.5907, 0.5952, 0.5878, 0.5731])
tensor([-0.7756, -0.7596, -0.7486, -0.7666, -0.8031])
tensor([-11., -11., -11., -11., -11.])
tensor([-11., -11., -11., -11., -11.])


In [14]:
print(out1/(2**out3[1]))

tensor([[[[-2.2829e+00,  8.8298e+00, -2.8543e+00,  ..., -1.2161e+01,
            4.7705e+00, -1.9948e+00],
          [-8.9168e+00, -5.4906e+00,  1.5604e+01,  ...,  4.1559e+00,
            2.3799e-02, -4.7651e+00],
          [ 7.4215e+00, -4.8867e+00, -4.6283e-01,  ..., -2.5203e+01,
            1.8020e+01, -2.9279e+00],
          ...,
          [-1.8889e+01, -2.5455e+00,  2.2238e+00,  ...,  1.7857e+01,
            4.1353e-01, -3.1454e+00],
          [-7.5872e+00,  1.3289e+01, -1.2501e+01,  ..., -1.6523e+01,
            1.3713e+01, -2.3464e+00],
          [-2.6401e+00, -2.6570e+01, -5.7005e+00,  ...,  1.6243e+01,
            1.4887e+00,  1.4540e+01]],

         [[-1.2053e+00, -9.7419e+00,  2.0468e+01,  ...,  1.2077e+01,
           -1.1452e+01,  1.5863e+01],
          [ 3.1031e+01,  8.8003e+00, -6.4341e+00,  ...,  2.0842e+01,
           -2.0963e+01,  1.3854e+01],
          [ 1.1019e+01, -6.0632e+00, -2.4715e+00,  ...,  3.0584e+00,
            1.4334e+01,  1.8386e+01],
          ...,
     